In [0]:
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt

### 1. Formulate the statement of the interpolation problem with Cubic Spline [mathematical formula]

We are given grid function $y_i=f(x_i)$ on grid $\Omega_n=\{x_i, i=\overline{0, n}\}$, where $x_i \in [a, b]$. Find such S $S_3(x_i)=f_i$ where

$\overline{S_3}(x) = \bigcup \limits_{i=0}^{n-1} \overline{S_{3, i}}(x)$ holds, such that $S_3 \in C^2[a, b]$, with defect $q = 1$

### 2. Formulate the functional and differential compatibility conditions [mathematical formula]

Compatibility conditions:
$\delta S^{(p_1)}_{3, i}(x_j) = S^{(p_1)}_{3, i}(x)$ $\large|_{x=x_i}$ $ - f^{(p_1)}(x)$ $\large |_{x=x_i}$ 
$ = 0, j = i, i+1$

Where
$S_{3, i} = a_{0, i} + a_{1, i} (x - x_i) + a_{2, i} (x - x_i)^2 + a_{3, i} (x - x_i)^3$, corresponding to $i$th spline link.


$p_1 = \{0, 2\}$ is picked 

Functional compatibility conditions (corresponding to the ends of $[x_i, x_{i+1}]$):

* $\delta S_{3, i}(x)$ $\large|_{x=x_i}$ $= S_{3, i}(x_i)$ $ - f(x_i)$ $ = 0$

* $\delta S_{3, i}(x)$ $\large|_{x=x_{i+1}}$ $= S_{3, i}(x_{i+1})$ $ - f(x_{i+1})$ $ = 0$

Differential compatibility conditions (corresponding to the ends of $[x_i, x_{i+1}]$):

* $\delta S^{''}_{3, i}(x)$ $\large|_{x=x_i}$ $= S^{''}_{3, i}(x_i)$ $ - f^{''}(x_i)$ $ = 0$

* $\delta S^{''}_{3, i}(x)$ $\large|_{x=x_{i+1}}$ $= S^{''}_{3, i}(x_{i+1})$ $ - f^{''}(x_{i+1})$ $ = 0$











### 3. Formulate stitching conditions [mathematical formula]

Stitching conditions: $S^{(p_2)}_{3, i-1}(x)$ $\large|_{x=x_i}$ 
$= S^{(p_2)}_{3, i}(x)$ $\large|_{x=x_i}$  $ i = \overline{1, n-1}$

$p_2\notin p_1$, where $p_1$ are for compatibility conditions.

$p_2 = {1}$ is picked.

* $S^{'}_{3, i-1}(x)$ $\large|_{x=x_i}$ 
$= S^{'}_{3, i}(x)$ $\large|_{x=x_i}$  $ i = \overline{1, n-1}$

### 4. Justify why these conditions provide you with the required smoothness [thesis text, no more than 500 characters]

Functional compatibility condition ensures that the resulting function solves the interpolation problem.

Differential compatibility condition provides additional equations in system to find coefficient of spline.

Stitching conditions ensures that the resulting function is continuous and differentiable on $[x_0;x_n]$.









### 5. Derive dependency formula: the dependence of the second derivatives at the grid nodes on the increment of the function (the function values difference on the grid nodes). [Mathematical formulas derivation. Detailed, with clear transitions]

Applying functional/differential conditions to the ends of this segment:
\begin{cases}\begin{align*}
S_{3, i}(x)|_{x=x_{i}} &= f(x_{i})\\ 
S_{3, i}(x)|_{x=x_{i+1}} &= f(x_{i+1})\\ 
S^{''}_{3, i}(x)|_{x=x_{i}} &= f^{''}(x_{i})\\ 
S^{''}_{3, i}(x)|_{x=x_{i+1}} &= f^{''}(x_{i+1})
\end{align*}\end{cases}

Lets find first and second derivatives of Cubic Spline:

\begin{align}
S^{'}_{3, i}(x) &= a_{1,i} + 2a_{2, i}(x - x_i) + 3a_{3, i}(x - x_i)^2\\
S^{''}_{3, i}(x) &= 2a_{2, i} + 6a_{3, i} (x - x_i)
\end{align}

Unfolding the equations:

\begin{cases}\begin{align*}  
a_{0,i} + a_{1,i}(x_i - x_i) + a_{2,i}(x_i - x_i)^2 + a_{3,i}(x_i - x_i)^3 &= f(x_i) \\ 
a_{0,i} + a_{1,i}(x_{i+1} - x_i) + a_{2,i}(x_{i+1} - x_i)^2 + a_{3,i}(x_{i+1} - x_i)^3 &= f(x_{i+1})\\ 
2a_{2,i} + 6a_{3,i}(x_i - x_i) &= f^{''}(x_i) \\ 
2a_{2,i} + 6a_{3,i}(x_{i+1} - x_i) &= f^{''}(x_{i+1}) 
\end{align*}\end{cases}

Reducing $x_i - x_i$:

\begin{cases}\begin{align*}  
a_{0,i} &= f(x_i) \\ 
a_{0,i} + a_{1,i}(x_{i+1} - x_i) + a_{2,i}(x_{i+1} - x_i)^2 + a_{3,i}(x_{i+1} - x_i)^3 &= f(x_{i+1})\\ 
2a_{2,i} &= f^{''}(x_i) \\ 
2a_{2,i} + 6a_{3,i}(x_{i+1} - x_i) &= f^{''}(x_{i+1}) 
\end{align*}\end{cases}

Let us have following replacings:
\begin{align}
h_{i+1} = x_{i+1} - x_{i}\\ 
\Delta f_i = f_{i+1} - f_{i}, f_i = f(x_i)\\ 
\Delta m_i = m_{i+1} - m_i, m_i = f^{''}(x_i)
\end{align}

In a shorter form it will look like:
\begin{cases}\begin{align*}  
a_{0,i} &= f_i \\ 
a_{0,i} + a_{1,i}\cdot h_{i+1} + a_{2,i}\cdot h_{i+1}^2 + a_{3,i}\cdot h_{i+1}^3 &= f_{i+1}\\ 
2a_{2,i} &= m_i \\ 
2a_{2,i} + 6a_{3,i}\cdot h_{i+1} &= m_{i+1} 
\end{align*}\end{cases}

After substitunting of third and fourth we will have:
\begin{align}
2a_{2,i} + 6a_{3,i}\cdot h_{i+1} - 2a_{2,i}= m_{i+1} - m_i &\Leftrightarrow \\
6a_{3,i}\cdot h_{i+1} = \Delta m_i &\Leftrightarrow \\
a_{3, i} = \frac{1}{6h_{i+1}}\Delta m_i
\end{align}

Also one can see that:
\begin{align}
2a_{2,i} = m_i &\Leftrightarrow \\
a_{2,i} = \frac{m_i}{2}
\end{align}

Putting it back to the place:
\begin{cases}\begin{align*}  
a_{0,i} &= f_i \\ 
a_{0,i} + a_{1,i}\cdot h_{i+1} + a_{2,i}\cdot h_{i+1}^2 + a_{3,i}\cdot h_{i+1}^3 &= f_{i+1}\\ 
a_{2,i} &= \frac{m_i}{2} \\ 
a_{3, i} &= \frac{1}{6h_{i+1}}\Delta m_i\\ 
\end{align*}\end{cases}

Replacing the second equation:
\begin{cases}\begin{align*}  
a_{0,i} &= f_i \\ 
f_i + a_{1,i}\cdot h_{i+1} + \frac{m_i}{2}\cdot h_{i+1}^2 + \frac{1}{6h_{i+1}}\Delta m_i\cdot h_{i+1}^3 &= f_{i+1}\\ 
a_{2,i} &= \frac{m_i}{2} \\ 
a_{3, i} &= \frac{1}{6h_{i+1}}\Delta m_i\\ 
\end{align*}\end{cases}

Working with the second equation:
\begin{align}
f_i + a_{1,i}\cdot h_{i+1} + \frac{m_i}{2}\cdot h_{i+1}^2 + \frac{1}{6h_{i+1}}\Delta m_i\cdot h_{i+1}^3 = f_{i+1} &\Leftrightarrow \\
f_i - f_{i+1} + a_{1,i}\cdot h_{i+1} + \frac{m_i}{2}\cdot h_{i+1}^2 + \frac{1}{6h_{i+1}}\Delta m_i\cdot h_{i+1}^3 = 0 &\Leftrightarrow \\
-\Delta f_i + a_{1,i}\cdot h_{i+1} + \frac{m_i}{2}\cdot h_{i+1}^2 + \frac{1}{6h_{i+1}}\Delta m_i\cdot h_{i+1}^3 = 0 &\Leftrightarrow \\
a_{1,i}\cdot h_{i+1}  = \Delta f_i - \frac{m_i}{2}\cdot h_{i+1}^2 - \frac{1}{6h_{i+1}}\Delta m_i\cdot h_{i+1}^3 &\Leftrightarrow \\
a_{1,i}\cdot h_{i+1}  = \Delta f_i - \frac{m_i}{2}\cdot h_{i+1}^2 - \frac{1}{6}\Delta m_i\cdot h_{i+1}^2 &\Leftrightarrow \\
a_{1,i}  = \Delta f_i - \frac{m_i}{2}\cdot h_{i+1} - \frac{1}{6}\Delta m_i\cdot h_{i+1}
\end{align}

Putting it back:
\begin{cases}\begin{align*}  
a_{0,i} &= f_i \\ 
a_{1,i} &= \Delta f_i - \frac{m_i}{2}\cdot h_{i+1} - \frac{1}{6}\Delta m_i\cdot h_{i+1}\\ 
a_{2,i} &= \frac{m_i}{2} \\ 
a_{3, i} &= \frac{1}{6h_{i+1}}\Delta m_i\\ 
\end{align*}\end{cases}

Now we have all $a$ and put it to the spline:
$$
S_{3, i}(x) = f_i + 
\left( \Delta f_i - \frac{m_i}{2}\cdot h_{i+1} - \frac{1}{6}\Delta m_i\cdot h_{i+1} \right)(x-x_i) + 
\frac{m_i}{2} (x - x_i)^2 + \frac{1}{6h_{i+1}}\Delta m_i (x - x_i)^3
$$

Rewriting stitching condition given with founded coefficients $a$:
\begin{align}
 S'_{i-1}(x_i) = S'_i(x_i) &\Leftrightarrow\\
S^{'}_{3, i}(x) = a_{1,i} + 2a_{2, i}(x - x_i) + 3a_{3, i}(x - x_i)^2 &\Leftrightarrow\\
 \left[\frac{1}{h_i} \Delta f_{i-1} - \frac{h_i}{2} m_{i-1} - \frac{h_i}{6} \Delta m_{i-1} +  m_{i-1}(x_i - x_{i-1}) + \frac{\Delta m_{i-1}}{2h_{i}}(x_i - x_{i-1})^2 = \\ =\frac{1}{h_{i+1}} \Delta f_{i} - \frac{h_{i+1}}{2} m_{i} - \frac{h_{i+1}}{6} \Delta m_{i} +  m_{i}(x_i - x_{i}) + \frac{\Delta m_{i}}{2h_{i+1}}(x_i - x_{i})^2 \right] &\Leftrightarrow\\
  \left[ \frac{1}{h_{i}} \Delta f_{i-1} - \frac{h_{i}}{2} m_{i-1} - \frac{h_{i}}{6} \Delta m_{i-1} +  m_{i-1} h_{i} + \frac{\Delta m_{i-1}}{2} h_{i} = \frac{1}{h_{i+1}} \Delta f_{i} - \frac{h_{i+1}}{2} m_{i} - \frac{h_{i+1}}{6} \Delta m_{i} \right] 
\end{align}

Getting rid of $\Delta m_{i-1}$ and $\Delta m_{i}$:
\begin{align}
\frac{h_i}{2}m_{i-1} + \frac{h_i}{3}(m_i - m_{i-1}) + \frac{h_{i+1}}{2}m_i + \frac{h_{i+1}}{6} (m_{i+1} - m_i) = \frac{1}{h_{i+1}} \Delta f_i  \frac{1}{h_i} \Delta f_{i-1} &\Leftrightarrow\\
\frac{h_i}{6}m_{i-1} + \frac{h_i + h_{i+1}}{3} m_i + \frac{h_{i+1}}{6} m_{i+1} =  \frac{1}{h_{i+1}} \Delta f_i - \frac{1}{h_i} \Delta f_{i-1}
\end{align}

### 6. Create a system of equations using this formula [Matrix representation. Mathematical formulas]

Tridiagonal matrix algorithm equations should be in form (p 31 from textbook):

$\alpha_i\cdot m_{i-1}-\beta_i \cdot m_i+\gamma_i \cdot m_{i+1}=\delta_i, \  \alpha_1=\gamma_n=0, \  i=\overline{1, n-1}$

These are:
\begin{align}
 \alpha_i &= \frac{h_i}{6} \\
 -\beta_i &= \frac{h_i + h_{i+1}}{3} \\
 \gamma_i &= \frac{h_{i+1}}{6} \\
 \delta_i &= \frac{\Delta f_i}{h_{i+1}} - \frac{\Delta f_{i-1}}{h_i}\\ 
\end{align}
\\

$$ \begin{bmatrix}
    -\beta_1    & \gamma_1    & 0 & 0     & \dots   & 0      \\
    \alpha_2    & -\beta_2    & \gamma_2  & 0  & \dots   & 0      \\
    0      & \alpha_3    & -\beta_3    & \gamma_3 & \dots   & 0      \\
    \vdots & \vdots & \vdots & \vdots & \ddots  & \vdots \\
    0      & 0      & 0      & \dots  & \alpha_{n-1}    & -\beta_{n-1} 
\end{bmatrix} \begin{bmatrix}
    m_1 \\
    m_2 \\
    m_3 \\
    \vdots \\
    m_{n-1}
\end{bmatrix} = \begin{bmatrix}
    \delta_1 \\
    \delta_2 \\
    \delta_3 \\
    \vdots \\
    \delta_{n-1}
\end{bmatrix} $$ 

### 7. Explain what is an unknown variable in this system. whether the system is closed with respect to an unknown variable. What is missing for closure. [Text, no more than 200 characters]

The unknown variable is $m_i$, system is not closed with respect to it. 

We need to add two more equations for natural spline for closure:
$m_0 = 0,\ m_N = 0$

### 8. Bring this matrix to the appropriate form to use the Tridiagonal matrix algorithm [Mathematical derivation. Use Gauss Elimination]

$$ \begin{bmatrix}
    1      & -P_1   & 0      & 0      & \dots     & 0      \\
    0      & 1      & -P_2   & 0      & \dots     & 0      \\
    0      & 0      & 1      & -P_3   & \dots     & 0      \\
    \vdots & \vdots & \vdots & \vdots & \ddots    & \vdots \\
    0      & 0      & 0      & 0      & \dots     & 1 
\end{bmatrix} M = \begin{bmatrix}
    Q_0 \\
    Q_1 \\
    Q_2 \\
    \vdots \\
    Q_{n-1}
\end{bmatrix} $$ 









### 9. Derive formulas of direct pass and reverse pass of Tridiagonal matrix algorithm [Mathematical formals]


* Forward pass:

  * Find initial $P$ and $Q$:
\begin{align}
 P_1 &= \frac{\gamma_1}{\beta_1} \\ 
 Q_1 &= - \frac{\delta_1}{\beta_1}
\end{align}
  * Find coefficients for $i=\overline{2, n-2}$:
\begin{align}
P_i &= \frac{\gamma_i}{\beta_i - \alpha_i P_{i-1}} \\ 
Q_i &= \frac{\alpha_i Q_{i-1} - \delta_i}{\beta_i - \alpha_i P_{i-1}} 
\end{align}

* Backward pass:

  * Find last $x$:
$$ x_{n-1} = \frac{\alpha_{n-1} Q_{n-2} - \delta_{n-1}}{\beta_{n-1} - \alpha_{n-1} P_{n-2}}  $$ 
  * Find $x_i$ for $i=\overline{1, n-2}$:
$$ x_i=P_i\cdot x_{i+1}+Q_i$$


### 10. Implement code prototype of the future algorithm implementation. Classes/methods (if you use OOP), functions. The final implementation (on language chosen by you) should not differ from the functions declared in the prototype. [Python code]

In [0]:
# M: Matrix in  tridiagonal form
# d: Corresponding vector

def thomas_alg(M, d):
  n = M.shape[0]
  x = [0] * n
  Pi = []
  Qi = []
  
  
  # Forward pass
  #   Initial P and Q
  Pi.append(M[0][1] / -M[0][0])
  Qi.append(d[0] / M[0][0])
  #   Finding coefficients
  for i in range(1, n):
    Pi.append(M[i][i+1] / (-M[i][i - 1] * Pi[-1] - M[i][i]))
    Qi.append((-d[i] + M[i][i - 1] * Qi[-1]) / (-M[i][i - 1] * Pi[-2] - M[i][i]))
  
  # Backward pass
  #   Last x
  x[n - 1] = (d[n - 1] - M[n - 1][n - 2] * Qi[-1]) / (M[n - 1][n - 2] + M[n - 1][n - 1])
  #   Finding x's
  for i in range(n - 2, 0, -1):
    x[i] = Pi[i] * x[i + 1] + Qi[i]
    
  return x
# x: array of solutions

### 11. Derive formula of Cubic Spline method error [Mathematical formulas]

Cubic Spline method error:

$|| f^{(p)}(x)-S_3^{(p)}(x)||_{C[a,b]} = \max_{[a,b]}|f^{(p)}(x)-S_3^{(p)}(x)|\leq M_4h^{4-p}$, $p=[0,2]$,

For each each p:

function:
$|| f^{(0)}(x)-S_3^{(0)}(x)||_{C[a,b]} = \max_{[a,b]}|f^{(0)}(x)-S_3^{(0)}(x)|\leq M_4h^{4}$, 

first derivative:
$|| f^{(1)}(x)-S_3^{(1)}(x)||_{C[a,b]} = \max_{[a,b]}|f^{(1)}(x)-S_3^{(1)}(x)|\leq M_4h^{3}$, 

second derivative: 
$|| f^{(2)}(x)-S_3^{(2)}(x)||_{C[a,b]} = \max_{[a,b]}|f^{(2)}(x)-S_3^{(2)}(x)|\leq M_4h^{2}$, 

This metrics only work if $f(x) \in C_4[a,b]$

### 12. Rate the complexity of the algorithm [Text, and rate in terms of big O, no more than 100 characters]



Forward pass is $\mathcal{O}(n)$ and backward pass is $\mathcal{O}(n)$ so overall complexity is $\mathcal{O}(n)$.


### Congrats!